Note: Don't worry about trying to understand the code. I'm hoping that this can help you to make the schedule quickier and avoid you having to double check your work as often. If you think this can helpful, then we can work to make it more detailed/useful.

In [1]:
import pandas as pd
import numpy as np

In [20]:
# I might be missing some activities
activities = ['swimming','boating','hockey','basketball','electives1','elective2','gymnastics','baseball','football',
              'soccer','tennis','lacrosse']
days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
periods = ['Period1','Period2','Period3','Period4','Period5','Period6','Period7']

In [21]:
boys_freshmen, boys_freshmen_df = {}, pd.DataFrame(columns = [day for day in days])
boys_cadets, boys_cadets_df = {}, pd.DataFrame(columns = [day for day in days])
boys_sophmore, boys_sophmore_df = {}, pd.DataFrame(columns = [day for day in days])
girls_freshmen, girls_freshmen_df = {}, pd.DataFrame(columns = [day for day in days])
girls_cadets, girls_cadets_df = {}, pd.DataFrame(columns = [day for day in days])
girls_sophmore, girls_sophmore_df = {}, pd.DataFrame(columns = [day for day in days])

groups = [boys_freshmen, boys_cadets, boys_sophmore, girls_freshmen, girls_cadets, girls_sophmore]

In [22]:
# Limit the number of times a group can have an activity during the week
# We can specify this to each group as well - i.e. boys more basketball, girls more gymnastics...
weekly_limits = {'swimming': 5,
                 'boating': 2,
                 'basketball': 3,
                 'baseball': 2,
                 'soccer': 2}

In [23]:
# This sets all activity periods to None
for group in groups:
    for day in days:
        group[day] = {}
        for period in periods:
            group[day][period] = None

In [24]:
# Here's a preview of that
boys_cadets

{'Friday': {'Period1': None,
  'Period2': None,
  'Period3': None,
  'Period4': None,
  'Period5': None,
  'Period6': None,
  'Period7': None},
 'Monday': {'Period1': None,
  'Period2': None,
  'Period3': None,
  'Period4': None,
  'Period5': None,
  'Period6': None,
  'Period7': None},
 'Saturday': {'Period1': None,
  'Period2': None,
  'Period3': None,
  'Period4': None,
  'Period5': None,
  'Period6': None,
  'Period7': None},
 'Sunday': {'Period1': None,
  'Period2': None,
  'Period3': None,
  'Period4': None,
  'Period5': None,
  'Period6': None,
  'Period7': None},
 'Thursday': {'Period1': None,
  'Period2': None,
  'Period3': None,
  'Period4': None,
  'Period5': None,
  'Period6': None,
  'Period7': None},
 'Tuesday': {'Period1': None,
  'Period2': None,
  'Period3': None,
  'Period4': None,
  'Period5': None,
  'Period6': None,
  'Period7': None},
 'Wednesday': {'Period1': None,
  'Period2': None,
  'Period3': None,
  'Period4': None,
  'Period5': None,
  'Period6': None,
  'P

In [25]:
# Before the schuedule is created, we can preset some values
# In this case, the first period on Wednesday is always 'Lazy Wednesday'
for group in groups:
    group['Wednesday']['Period1'] = 'Lazy Wednesday'

In [7]:
# Before an activity is added to a period the following conditions must be satisfied 
def check_conditions(activity, group, day, period):
    condition = True
    
    # No two groups can have the same activity at the same time
    for other_groups in groups:
        if other_groups[day][period] == activity:
            condition = False

    # A group can't have an activity at the same time during the week
    # i.e. If the cadet boys have soccer Monday, Period 1, 
    #      then they will not have it during Period 1 for the rest of the week
    for other_days in days:
        if group[other_days][period] == activity:
            condition = False
            
    # A group can't have the same activity twice in a day
    for other_periods in periods:
        if group[day][other_periods] == activity:
            condition = False

    # Limits an activity to its weekly limit
    if activity in weekly_limits:
        count = 1
        for other_days in days:
            for other_periods in periods:
                if group[other_days][other_periods] == activity:
                    count += 1
        if count > weekly_limits[activity]:
            condition = False
        
    # Can't have boating and swimming on the same day
    if activity in ['swimming', 'boating']:
        for other_periods in periods:
            if group[day][other_periods] == 'swimming' or group[day][other_periods] == 'boating':
                condition = False
        
    return condition

# Note: These rules are simplified, we can make more exact ones later.

In [8]:
# Assigns the activities to the periods
def add_activities():
    for activity in activities:
        for group in groups:
            for day in days:
                for period in periods:
                    if group[day][period] == None:
                        if check_conditions(activity, group, day, period):
                            group[day][period] = activity

In [9]:
add_activities()

In [10]:
boys_freshmen_df = boys_freshmen_df.from_dict(boys_freshmen)[[day for day in days]]
boys_cadets_df = boys_cadets_df.from_dict(boys_cadets)[[day for day in days]]
boys_sophmore_df = boys_sophmore_df.from_dict(boys_sophmore)[[day for day in days]]
girls_freshmen_df = girls_freshmen_df.from_dict(girls_freshmen)[[day for day in days]]
girls_cadets_df = girls_cadets_df.from_dict(girls_cadets)[[day for day in days]]
girls_sophmore_df = girls_sophmore_df.from_dict(girls_sophmore)[[day for day in days]]

In [11]:
used_activities = {}
unused_activities = {}
for day in days:
    used_activities[day] = {}
    unused_activities[day] = {}
    for period in periods:
        used_activities[day][period] = []
        unused_activities[day][period] = []

In [12]:
for group in groups:
    for day in days:
        for period in periods:
            used_activities[day][period].append(group[day][period])

In [13]:
for day in days:
    for period in periods:
        for activity in activities:
            if activity not in used_activities[day][period]:
                unused_activities[day][period].append(activity)

In [14]:
# This shows all the activities that are not in use during a period
unused_activities_df = pd.DataFrame().from_dict(unused_activities)
pd.option_context('display.max_rows', None, 'display.max_columns', None)
unused_activities_df

,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
Period1,"[boating, basketball, gymnastics, baseball, te...","[boating, baseball, football, soccer, tennis, ...","[basketball, electives1, baseball, soccer, ten...","[swimming, basketball, baseball, soccer, tenni...","[gymnastics, baseball, football, soccer, tenni...","[boating, baseball, football, soccer, tennis, ...","[swimming, boating, hockey, basketball, electi..."
Period2,"[boating, basketball, baseball, soccer, tennis...","[boating, baseball, football, soccer, tennis, ...","[basketball, electives1, baseball, soccer, ten...","[swimming, hockey, basketball, baseball, socce...","[boating, baseball, football, soccer, tennis, ...","[boating, baseball, football, soccer, tennis, ...","[elective2, gymnastics, baseball, soccer, tenn..."
Period3,"[boating, basketball, electives1, baseball, so...","[boating, elective2, gymnastics, soccer, tenni...","[swimming, hockey, basketball, baseball, footb...","[swimming, basketball, electives1, baseball, s...","[boating, basketball, baseball, football, tenn...","[boating, elective2, gymnastics, soccer, tenni...","[boating, baseball, football, soccer, tennis, ..."
Period4,"[swimming, boating, hockey, baseball, tennis, ...","[boating, elective2, gymnastics, soccer, tenni...","[swimming, hockey, basketball, baseball, footb...","[swimming, hockey, basketball, electives1, gym...","[boating, basketball, baseball, football, tenn...","[boating, elective2, gymnastics, soccer, tenni...","[boating, baseball, football, soccer, tennis, ..."
Period5,"[boating, basketball, baseball, football, socc...","[boating, basketball, electives1, soccer, tenn...","[swimming, boating, basketball, elective2, bas...","[swimming, basketball, electives1, elective2, ...","[swimming, boating, hockey, basketball, baseba...","[boating, basketball, electives1, soccer, tenn...","[boating, elective2, gymnastics, baseball, ten..."
Period6,"[boating, basketball, baseball, football, socc...","[boating, basketball, electives1, soccer, tenn...","[swimming, boating, basketball, elective2, gym...","[swimming, basketball, elective2, baseball, fo...","[swimming, boating, hockey, basketball, gymnas...","[boating, basketball, electives1, soccer, tenn...","[swimming, boating, hockey, soccer, tennis, la..."
Period7,"[boating, basketball, electives1, elective2, b...","[swimming, boating, hockey, soccer, tennis, la...","[swimming, boating, basketball, baseball, foot...","[swimming, boating, basketball, gymnastics, ba...","[boating, basketball, electives1, elective2, b...","[swimming, boating, hockey, soccer, tennis, la...","[boating, basketball, electives1, baseball, te..."


In [19]:
# Here is an example of a weekly schedule
# Some periods say 'None' - this is because I haven't made the code good enough yet
boys_sophmore_df

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
Period1,basketball,electives1,Lazy Wednesday,swimming,hockey,elective2,football
Period2,electives1,basketball,swimming,hockey,gymnastics,football,elective2
Period3,swimming,hockey,basketball,electives1,elective2,boating,gymnastics
Period4,hockey,swimming,electives1,elective2,soccer,gymnastics,boating
Period5,baseball,football,hockey,gymnastics,electives1,soccer,None
Period6,football,baseball,elective2,tennis,lacrosse,None,hockey
Period7,elective2,gymnastics,football,lacrosse,swimming,hockey,electives1


Note: Let me know what you think. Also, don't worry if you think this will create 'work' for me. I like doing this sort of thing and helping you guys out.